# Projecte github

# Imports

In [24]:
# Draw the plots immediately after the current cell
%matplotlib inline

import pandas as pd
# pip install PyMySQL
import pymysql
import warnings

# Project configuration

The file that we are going to use to set up the basic structure:

GitHubScraper/JupyterNotebook/Structure/githubProjectStructure.sql

from the github repository.

### Step 1, import the structure
In workbench: 
"Server" -> "Data Import", and then you should select the direction of "githubProjectStructure.sql" using the 3 dots and import from self-contained file like in the following image:

![Javatpoint](./Images/Project_Setup/data_import.png)  

Then scroll down, and create a new Schema by clicking on the "New..." button. Name it: "githubProject".

![Javatpoint](./Images/Project_Setup/create_new_schema.png)  

Scroll to the bottom, and in the bottom right, click on "Start import". If there are no errors, it will transitionate to "Import progress" and indicate success.

Refresh the scehmas:

![Javatpoint](./Images/Project_Setup/refresh.png)  

### Step 2, importing the data from the csv files

Download the csv files [here](https://drive.google.com/drive/folders/1NWhfFss0_M9V_clkcE9TH-Fy3oIEndWV?usp=sharing).

Selecting the "githubProject" scheme, right click and "Table Data Import Wizard", with this, we are going to import each csv file to the matching table.

You have to select the .csv you want to import, for example in this case, the trendVisits.csv

![Javatpoint](./Images/Project_Setup/file_import.png) 

Then, since you already have the structure, you have to use the existing table that matches with the correct csv file:

![Javatpoint](./Images/Project_Setup/use_existing.png) 

After that, check that the variables type are correct and import it. Do this for each csv file.

You can also verify the data by doing this:

![Javatpoint](./Images/Project_Setup/verify.png) 



# Introduction

TODO index or sections indicating what are we doing...

In [16]:
#https://www.geeksforgeeks.org/connect-to-mysql-using-pymysql-in-python/
# Connect to MySQL database
def tryToConnectMySQL(db_name, db_host, db_port, db_username, db_password):
    try:
        conn = pymysql.connect(host=db_host,
                               port=db_port,
                               user=db_username,
                               password=db_password,
                               db=db_name)
        if conn:
            print("Connection successful")
        else:
            print("Error")
        
        return conn
        
    except pymysql.Error as e:        
        warnings.warn("Error connecting to MySQL:", e)
        return None

# Pass arguments from outside
db_name = "githubProject"
db_host = "localhost"
db_port = 3306
db_username = "root"
db_password = input("Input your password")

dataBaseConnection = tryToConnectMySQL(db_name, db_host, db_port, db_username, db_password)

Input your password Muck23Luck_


Connection successful


In [17]:
# Execute query and return data in pandas dataframe 
def execute_select_query(cursor, query):
    try:
        cursor.execute(query)
        output = cursor.fetchall()
        # Fetch column names from cursor's description
        columns = [desc[0] for desc in cursor.description]
        
        # Convert output to pandas DataFrame
        if output:
            df = pd.DataFrame(output, columns=columns)
            print("Query executed successfully!")
        return output, df
            
    except pymysql.Error as e:
        print("Error executing query:")
        warnings.warn(str(e))
        return None, None

In [18]:
# Obtain the cursos in order to interact with the DataBase
cursor = dataBaseConnection.cursor()

# Tables
Shows the table for the different entities of the Data Base.

## Repositories

In [21]:
query = "SELECT * FROM Repositories;"

output_repo, df_repo = execute_select_query(cursor, query)
df_repo.tail(5)

Query executed successfully!


,owner,name,mainLanguage,license
1204,ujjwalkarn,Twitter-Sentiment-Analysis,Python,
1205,resemble-ai,resemble-enhance,Python,
1206,resemble-ai,MelNet,Python,
1207,resemble-ai,resemble-unity-text-to-speech,C#,
1208,dennybritz,reinforcement-learning,Jupyter Notebook,


In [7]:
query = "SELECT * FROM RepositoryVisits;"

output_repo_visists, df_repo_visists = execute_select_query(cursor, query)
df_repo_visists.head(5)

Query executed successfully!


,date,name,owner,forks,commits,stars,watchers,contributors,openIssues,closedIssues,openPullRequests,closedPullRequests
0,1.709455e+09,go-gitea,gitea,5100,17384,-1,-1,-1,-1,-1,-1,-1
1,1.709455e+09,colbyfayock,next-wordpress-starter,279,772,-1,-1,-1,-1,-1,-1,-1
2,1.709455e+09,mattermost,mattermost,6700,18212,-1,-1,-1,-1,-1,-1,-1
3,1.709455e+09,Luohuayu,CatServer,194,-1,-1,-1,-1,-1,-1,-1,-1
4,1.709455e+09,hoppscotch,hoppscotch,4100,5164,-1,-1,-1,-1,-1,-1,-1


## Owners

In [8]:
query = "SELECT * FROM Owners;"
# Use _ to ignore
_, df_owners = execute_select_query(cursor, query)
df_owners.head(5)

Query executed successfully!


,username
0,PinewoodPip
1,go-gitea
2,colbyfayock
3,mattermost
4,Luohuayu


In [9]:
query = "SELECT * FROM OwnerVisits;"

output_owner_visits, df_owner_visits = execute_select_query(cursor, query)
df_owner_visits.head(5)

Query executed successfully!


,date,username,contributionsLastYear
0,1.709455e+09,PinewoodPip,1774
1,1.709455e+09,colbyfayock,1878
2,1.709455e+09,Luohuayu,79
3,1.709455e+09,jacopotediosi,376
4,1.709455e+09,trekhleb,210


## Commits

In [10]:
query = "SELECT * FROM Commits;"

output_commits, df_commits = execute_select_query(cursor, query)
df_commits.head(5)

Query executed successfully!


,sha,author,repository,repositoryOwner,message
0,d91358e1238066650bb7d1b271dfcfc8ca6c2026,lukekarrys,rfcs,npm,chore: postinstall for dependabot template-oss PR
1,149ad1d4d51463b0a2c6902a5f133fe9dcbdd705,dependabot[bot],rfcs,npm,chore: bump @npmcli/template-oss from 4.21.1 t...
2,c452b65abf0af780e9a2b6f0e0b00fb0aa6931e7,lukekarrys,rfcs,npm,chore: postinstall for dependabot template-oss PR
3,b4b6622cca2603ae7ced4bc7f3f2ff2b7ee5f291,dependabot[bot],rfcs,npm,chore: bump @npmcli/template-oss from 4.19.0 t...
4,bdf4764322ce20137975298c8283b52f08549b37,lukekarrys,rfcs,npm,chore: postinstall for dependabot template-oss PR


## Topics

In [11]:
query = "SELECT * FROM Topics;"

output_topics, df_topics = execute_select_query(cursor, query)
df_topics.tail(5)

Query executed successfully!


,name,mainLanguage
9,mod,Java
10,unity3d,C#
11,machine-learning,Jupyter Notebook
12,deep-learning,Python
13,emulation,C++


In [12]:
query = "SELECT * FROM TopicVisits;"

_, df_topic_visits = execute_select_query(cursor, query)
df_topic_visits.head(5)

Query executed successfully!


,date,name,repositories,followers
0,1.709539e+09,nodejs,238297,77600
1,1.709539e+09,javascript,510373,169000
2,1.709539e+09,npm,19224,28800
3,1.709539e+09,next,4376,1500
4,1.709539e+09,react,347483,98800


## Trend

In [13]:
query = "SELECT * FROM TrendVisits;"

_, df_trend_visits = execute_select_query(cursor, query)
df_trend_visits.head(5)

Query executed successfully!


,date,repo_name,owner,starsToday
0,1.712734e+09,plandex-ai,plandex,1459
1,1.712734e+09,nashsu,FreeAskInternet,1284
2,1.712734e+09,drawdb-io,drawdb,665
3,1.712734e+09,langgenius,dify,581
4,1.712734e+09,PawanOsman,ChatGPT,410


In [14]:
# Closes the connection
dataBaseConnection.close() 